In [2]:
#!python -m pip install featuretools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.5/522.5 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.5/230.5 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.1/930.1 kB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 83.3 MB/s eta 0:00:00ta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for PyMeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=731999 sha256=90c97ed91cd8881345cf22ef9dc7af1785732ff22eb9f12e2014a6500706dd06
  Stored in directory: /opt/ml/.cache/pip/wheels/b9/49/19/1ec4bcf2cc76

In [2]:
import torch
import pandas as pd
import numpy as np
# pip install transformers
from transformers import BertTokenizer, BertModel
# 만약 주피터 노트북에서 아래와 관계있는 에러가 발생한다면
# pip install ipywidgets

In [9]:
main_df = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv') # user-item-time
title_df = pd.read_csv('/opt/ml/input/data/train/titles.tsv', sep='\t') # item-title
year_df = pd.read_csv('/opt/ml/input/data/train/years.tsv', sep='\t') # item-year
director_df = pd.read_csv('/opt/ml/input/data/train/directors.tsv', sep='\t') # item-director
genre_df = pd.read_csv('/opt/ml/input/data/train/genres.tsv', sep='\t') # item-genre(name)
writer_df = pd.read_csv('/opt/ml/input/data/train/writers.tsv', sep='\t') # item-writer

In [5]:
writer_received_df = pd.read_csv('/opt/ml/input/fighting/FE/writer/writer_max_sum_intercnt.csv')

In [6]:
writer_received_df

,writer,wri_max_cnt,wri_sum_cnt
0,nm0000005,1383,6020
1,nm0000019,905,3531
2,nm0000033,1190,1787
3,nm0000036,813,1574
4,nm0000040,9155,48618
...,...,...,...
2985,nm5335213,821,1107
2986,nm5371819,796,1910
2987,nm5927607,5118,5171
2988,nm5927608,5118,5171


In [7]:
writer_df.shape
writer_df.head(3)

,item,writer
0,1237,nm0000005
1,5147,nm0000005
2,7327,nm0000005


In [10]:
# writer 결측 item 작가를 'nm0000000'로 채워주기
no_wri_item_list = list(set(main_df['item']) - set(writer_df['item']))
no_wri_item_df = pd.DataFrame([x for x in zip(no_wri_item_list, ['nm0000000']*len(no_wri_item_list))])
no_wri_item_df.columns=writer_df.columns # 컬럼명 동일하게
writer_df = pd.concat([writer_df, no_wri_item_df]) # 기존 writer_df 뒤에 작가없는 영화 추가

In [11]:
writer_df

,item,writer
0,1237,nm0000005
1,5147,nm0000005
2,7327,nm0000005
3,2068,nm0000005
4,7396,nm0000005
...,...,...
1154,55284,nm0000000
1155,81910,nm0000000
1156,4091,nm0000000
1157,2046,nm0000000


In [12]:
item_list = list(writer_df['item'].unique())

In [13]:
df = writer_df.merge(writer_received_df, how='left', on='writer')
df

,item,writer,wri_max_cnt,wri_sum_cnt
0,1237,nm0000005,1383,6020
1,5147,nm0000005,1383,6020
2,7327,nm0000005,1383,6020
3,2068,nm0000005,1383,6020
4,7396,nm0000005,1383,6020
...,...,...,...,...
12460,55284,nm0000000,816,816
12461,81910,nm0000000,816,816
12462,4091,nm0000000,816,816
12463,2046,nm0000000,816,816


In [14]:
# 작가가 'nm0000000'인 경우는 결측치를 채워준 부분이므로, 각 영화를 독립적으로 본다.
# 아래의 코드들은 'nm0000000'인 경우를 처리하기 위한 코드이다.

In [15]:
item_popularity = pd.read_csv('/opt/ml/input/fighting/FE/item/item_interaction_cnt.csv')

In [16]:
df[df['writer']=='nm0000000']

,item,writer,wri_max_cnt,wri_sum_cnt
11306,26629,nm0000000,816,816
11307,40966,nm0000000,816,816
11308,4104,nm0000000,816,816
11309,77833,nm0000000,816,816
11310,83976,nm0000000,816,816
...,...,...,...,...
12460,55284,nm0000000,816,816
12461,81910,nm0000000,816,816
12462,4091,nm0000000,816,816
12463,2046,nm0000000,816,816


In [17]:
df = df.merge(item_popularity, how='left', on='item')
df

,item,writer,wri_max_cnt,wri_sum_cnt,item_cnt
0,1237,nm0000005,1383,6020,1383
1,5147,nm0000005,1383,6020,949
2,7327,nm0000005,1383,6020,751
3,2068,nm0000005,1383,6020,502
4,7396,nm0000005,1383,6020,335
...,...,...,...,...,...
12460,55284,nm0000000,816,816,207
12461,81910,nm0000000,816,816,56
12462,4091,nm0000000,816,816,204
12463,2046,nm0000000,816,816,845


In [18]:
df.loc[df['writer']=='nm0000000', 'wri_max_cnt'] = df.loc[df['writer']=='nm0000000']['item_cnt']
df.loc[df['writer']=='nm0000000', 'wri_sum_cnt'] = df.loc[df['writer']=='nm0000000']['item_cnt']

In [19]:
df

,item,writer,wri_max_cnt,wri_sum_cnt,item_cnt
0,1237,nm0000005,1383,6020,1383
1,5147,nm0000005,1383,6020,949
2,7327,nm0000005,1383,6020,751
3,2068,nm0000005,1383,6020,502
4,7396,nm0000005,1383,6020,335
...,...,...,...,...,...
12460,55284,nm0000000,207,207,207
12461,81910,nm0000000,56,56,56
12462,4091,nm0000000,204,204,204
12463,2046,nm0000000,845,845,845


In [20]:
df = df.drop(columns=['item_cnt'], axis=1)

In [21]:
df

,item,writer,wri_max_cnt,wri_sum_cnt
0,1237,nm0000005,1383,6020
1,5147,nm0000005,1383,6020
2,7327,nm0000005,1383,6020
3,2068,nm0000005,1383,6020
4,7396,nm0000005,1383,6020
...,...,...,...,...
12460,55284,nm0000000,207,207
12461,81910,nm0000000,56,56
12462,4091,nm0000000,204,204
12463,2046,nm0000000,845,845


In [ ]:
# 이제 각 아이템별 기여한 작가 중 가장 인기많은 작가가 받은 interaction 수를 구할 것이다.

In [22]:
item_grouped = df.groupby('item')

In [23]:
result = item_grouped.max()

In [24]:
result = result.reset_index()
result = result.drop(columns=['writer'], axis=1)

In [25]:
result

,item,wri_max_cnt,wri_sum_cnt
0,1,12217,60006
1,2,5769,15224
2,3,891,3267
3,4,8908,14462
4,5,3856,7519
...,...,...,...
6802,118700,54,54
6803,118900,4970,9706
6804,118997,236,288
6805,119141,4688,7135


In [27]:
result.to_csv('item_popular_wri_max_sum_cnt.csv', index=False)